This notebook serves as an example of how to load and manipulate the [COMP6 dataset](https://github.com/isayev/COMP6) using a `Dataset` object.

In [ ]:
import os
import numpy as np

from ase import Atoms

# Initialize the database

In [ ]:
from colabfit.tools.database import MongoDatabase, load_data

client = MongoDatabase('colabfit_database', nprocs=6)#, drop_database=True)

# Setup

The COMP6 dataset is one of a collection of datasets that uses the [ANI-1 format](https://github.com/isayev/ANI1_dataset) for loading. Before running this example, you should make sure that [pyanitools.py](https://github.com/isayev/ANI1_dataset/blob/master/readers/lib/pyanitools.py) is in `PYTHONPATH` so that you can use it for loading from the ANI-formatted HDF5 files.

In [ ]:
import sys

my_path_to_pyanitools = '../../../svreg_data/AlZnMg/AL_Al/'
sys.path.append(my_path_to_pyanitools)

# Custom reader

Since COMP6 is not stored in one of the core file formats, a user-specified `reader` function must be provided to `load_data` in order to read the data.

In [ ]:
def reader(path):    
    import pyanitools as pya
    
    adl = pya.anidataloader(path)
    
    images = []
    for data in adl:        
        for i in range(data['coordinates'].shape[0]):
            atoms = Atoms(symbols=data['species'], positions=data['coordinates'][i])
            
            atoms.info['name'] = '{}_{}_conformer_{}'.format(os.path.split(path)[-1], data['path'], i)
            
            atoms.info['energy'] = data['energies'][i]
            atoms.arrays['forces'] = data['forces'][i]
            
            atoms.info['cm5'] = data['cm5'][i]
            atoms.info['hirdipole'] = data['hirdipole'][i]
            atoms.info['hirshfeld'] = data['hirshfeld'][i]
            atoms.info['spindensities'] = data['spindensities'][i]
            
            images.append(atoms)
            
    return images

# Data loading

In [ ]:
comp6_property_definition = {
    'property-id': 'comp6-data',
    'property-title': 'cm5, hirdipole, hirshfeld, spindensities',
    'property-description': 'Charges, dipoles, and spin densities',
    'energy':       {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Configuration energy'},
    'forces':       {'type': 'float', 'has-unit': True, 'extent': [":", 3], 'required': True, 'description': 'Atomic forces'},
    'cm5':          {'type': 'float', 'has-unit': True, 'extent': [":"], 'required': True, 'description': 'CM5 atomic charges'},
    'hirshfeld':    {'type': 'float', 'has-unit': True, 'extent': [":"], 'required': True, 'description': 'Hirshfeld atomic charges'},
    'hirdipole':    {'type': 'float', 'has-unit': True, 'extent': [":", 3], 'required': True, 'description': 'Hirshfeld atomic dipoles'},
}

In [ ]:
client.insert_property_definition(comp6_property_definition)

In [ ]:
client.get_property_definition('comp6-data')

In [ ]:
property_map = {
    'comp6-data': {
        # Property Definition field: {'field': ASE field, 'units': ASE-readable units}
        'energy':    {'field': 'energy',    'units': 'kcal/mol'},
        'forces':    {'field': 'forces',    'units': 'kcal/mol/Ang'},
        'cm5':       {'field': 'cm5',       'units': 'elementary_charge'},
        'hirshfeld': {'field': 'hirshfeld', 'units': 'elementary_charge'},
        'hirdipole': {'field': 'hirdipole', 'units': 'elementary_charge*Ang'},
    }
}

In [ ]:
images = load_data(
    file_path='../../../colabfit/data/isayev/COMP6/COMP6v1/',
    file_format='folder',
    name_field='name',  # key in Configuration.info to use as the Configuration name
    elements=['C', 'H', 'N', 'O'],    # order matters for CFG files, but not others
    default_name='comp6',  # default name with `name_field` not found
    reader=reader,
    glob_string='*.h5',
    verbose=True
)

In [ ]:
from colabfit.tools.property_settings import PropertySettings

pso = PropertySettings(
    method='Gaussian09',
    description='COMP6 property settings calculation',
    files=[],
    labels=['DFT', 'wb97x', '6-31G(d)'],
)

ids = client.insert_data(
    images,
    property_map=property_map,
    property_settings={'comp6-data': pso},
#     generator=False,
    verbose=True
)

In [ ]:
configuration_set_regexes = {
    'ani_md_bench':
        'Forces from the ANI-1x potential are applied to run '\
        '1 ns of vacuum molecular dynamics with a 0.25 fs time '\
        'step at 300 K using the Langevin thermostat on 14 well-'\
        'known drug molecules and two small proteins. System '\
        'sizes range from 20 to 312 atoms. A random subsample '\
        'of 128 frames from each 1 ns trajectory is selected, and '\
        'reference DFT single point calculations are performed '\
        'to obtain QM energies and forces.',
    'drugbank_testset':
        'This benchmark is developed '\
        'through a subsampling of the DrugBank database '\
        'of real drug molecules. 837 SMILES strings con'\
        'taining C, N, and O are randomly selected. Like the '\
        'GDB7to9 benchmark, the molecules are embedded in '\
        '3D space, structurally optimized, and normal modes are '\
        'computed. DNMS is utilized to generate random '\
        'non-equilibrium conformations.',
    'gdb11_0[7-9]':
        'The GDB-11 subsets contain'\
        'ing 7 to 9 heavy atoms (C, N, and O) are subsampled '\
        'and randomly embedded in 3D space using RDKit '\
        '[www.rdkit.org]. A total of 1500 molecule SMILES '\
        '[opensmiles.org] strings are selected: 500 per 7, 8, '\
        'and 9 heavy-atom sets. The resulting structures are '\
        'optimized with tight convergence criteria, and nor'\
        'mal modes/force constants are computed using the '\
        'reference DFT model. Finally, diverse normal mode '\
        'sampling (DNMS) is carried out to generate non-'\
        'equilibrium conformations.',
    'gdb1[1,3]_1[0-3]':
        'GDB10to13 benchmark. Subsamples of 500 SMILES '\
        'strings each from the 10 and 11 heavy-atom subsets '\
        'of GDB-11 and 1000 SMILES strings from the 12 '\
        'and 13 heavy-atom subsets of the GDB-13 database '\
        'are randomly selected. DNMS is utilized to generate '\
        'random non-equilibrium conformations.',
    'tripeptide_full':
        'Tripeptide benchmark. 248 random tripeptides contain'\
        'ing H, C, N, and O are generated using FASTA strings '\
        'and randomly embedded in 3D space using RDKit. As '\
        'with GDB7to9, the molecules are optimized, and nor'\
        'mal modes are computed. DNMS is utilized to generate '\
        'random non-equilibrium conformations.',
    's66x8_wb97x6-31gd':
        'S66x8 benchmark. This dataset is built from the '\
        'original S66x850 benchmark for comparing accuracy '\
        'between different methods in describing noncovalent '\
        'interactions common in biological molecules. S66x8 is '\
        'developed from 66 dimeric systems involving hydro'\
        'gen bonding, pi-pi stacking, London interactions, and '\
        'mixed influence interactions. While the keen reader '\
        'might question the use of this benchmark without dis'\
        'persion corrections, since dispersion corrections such '\
        'as the D362 correction by Grimme et al. are a posteriori '\
        'additions to the produced energy, then a comparison '\
        'without the correction is equivalent to a comparison '\
        'with the same dispersion corrections applied to both '\
        'models.'
}

In [ ]:
cs_ids = []

for i, (regex, desc) in enumerate(configuration_set_regexes.items()):
    co_ids = client.get_data(
        'configurations',
        fields='_id',
        query={'names': {'$regex': regex}},
        ravel=True
    ).tolist()
    
    print(f'Configuration set {i}', f'({regex}):'.rjust(20), f'{len(co_ids)}'.rjust(7))

    cs_id = client.insert_configuration_set(co_ids, description=desc, verbose=True)
    
    cs_ids.append(cs_id)

In [ ]:
all_co_ids, all_pr_ids = list(zip(*ids))
len(all_pr_ids)

In [ ]:
ds_id = client.insert_dataset(
    cs_ids=cs_ids,
    pr_ids=all_pr_ids,
    name='COMP6',
    authors=[
        'Justin S. Smith',
        'Ben Nebgen',
        'Nicholas Lubbers',
        'Olexandr Isayev',
        'Adrian E. Roitberg'
    ],
    links=[
        'https://aip.scitation.org/doi/full/10.1063/1.5023802',
        'https://github.com/isayev/COMP6',
    ],
    description='This repository contains the COMP6 benchmark '\
        'for evaluating the extensibility of machine-learning '\
        'based molecular potentials.',
    resync=True,
    verbose=True,
)
ds_id

In [ ]:
ds = client.get_dataset(ds_id, verbose=True)['dataset']

In [ ]:
for k,v in ds.aggregated_info.items():
    print(k, v)

# Exploration

In [ ]:
ds.aggregated_info['property_fields']

In [ ]:
from IPython.display import Image

In [ ]:
fig = client.plot_histograms(ds.aggregated_info['property_fields'], ids=ds.property_ids, verbose=True)

In [ ]:
Image(fig.to_image(format="png", width=800, height=500, scale=1))

# Filtering

In [ ]:
clean_config_sets, clean_property_ids = client.filter_on_properties(
    ds_id=ds_id,
    filter_fxn=lambda p: p['comp6-data']['energy']['source-value'] > -4000,
    fields=['comp6-data.energy'],
    verbose=True
)

In [ ]:
clean_cs_ids = []

for cs in clean_config_sets:
    cs_id = client.insert_configuration_set(cs.configuration_ids, description=cs.description, verbose=True)
    
    clean_cs_ids.append(cs_id)

In [ ]:
clean_ds_id = client.insert_dataset(
    cs_ids=clean_cs_ids,
    pr_ids=clean_property_ids,
    name='COMP6_filtered',
    authors=ds.authors,
    links=ds.links,
    description="A filtered version of the COMP6 dataset "\
    "that removed all configurations with energies < -4000",
    resync=True,
    verbose=True
)
clean_ds_id

In [ ]:
clean_ds = client.get_dataset(clean_ds_id, verbose=True)['dataset']

In [ ]:
fig = client.plot_histograms(clean_ds.aggregated_info['property_fields'], ids=clean_ds.property_ids, verbose=True)

In [ ]:
Image(fig.to_image(format="png", width=800, height=500, scale=1))

## Extracting data from a single configuration set

In [ ]:
cs = client.get_configuration_set(clean_ds.configuration_set_ids[0])['configuration_set']

pr_ids = client.get_data(
    'configurations',
    fields='relationships.properties',
    ids=cs.configuration_ids,
    ravel=True
).tolist()

len(pr_ids)

In [ ]:
fig = client.plot_histograms(
    clean_ds.aggregated_info['property_fields'],
    ids=pr_ids,
    verbose=True
)

Image(fig.to_image(format="png", width=800, height=500, scale=1))